In [7]:
#r "nuget: Microsoft.ML"
using Microsoft.ML;
using Microsoft.ML.Data;

Installing package Microsoft.ML........

This example comes from the ML.NET documentation: https://docs.microsoft.com/en-us/dotnet/api/microsoft.ml.conversionsextensionscatalog.mapkeytovalue?view=ml-dotnet

In [8]:
class DataPoint
{
    public string Label { get; set; }
    [VectorType(10)]
    public float[] Features { get; set; }
}

class TransformedData
{
    public uint Label { get; set; }
    public uint PredictedLabel { get; set; }
    public string LabelOriginalValue { get; set; }
    public string PredictedLabelOriginalValue { get; set; }
}

In [9]:
static List<DataPoint> GenerateRandomDataPoints(int count, int featureVectorLenght)
{
    var examples = new List<DataPoint>();
    var rnd = new Random(0);
    for (int i = 0; i < count; ++i)
    {
        var example = new DataPoint();
        example.Features = new float[featureVectorLenght];
        var res = i % 4;
        // Generate random float feature values.
        for (int j = 0; j < featureVectorLenght; ++j)
        {
            var value = (float)rnd.NextDouble() + res * 0.2f;
            example.Features[j] = value;
        }

        // Generate label based on feature sum.
        if (res == 0)
            example.Label = "AA";
        else if (res == 1)
            example.Label = "BB";
        else if (res == 2)
            example.Label = "CC";
        else
            example.Label = "DD";
        examples.Add(example);
    }
    return examples;
}

In [10]:
var mlContext = new MLContext(seed: 0);

In [11]:
var examples = GenerateRandomDataPoints(1000, 10);

In [12]:
var dataView = mlContext.Data.LoadFromEnumerable(examples);

In [14]:
var pipeline = mlContext
    // Convert the string labels into key types.
    .Transforms.Conversion.MapValueToKey("Label")
    // Apply StochasticDualCoordinateAscent multiclass trainer.
    .Append(mlContext.MulticlassClassification.Trainers.SdcaMaximumEntropy());

In [15]:
var dataWithPredictions = pipeline.Fit(dataView).Transform(dataView);

At this point, the Label column is transformed from strings, to DataViewKeyType and the transformation has added the PredictedLabel column, with same DataViewKeyType as transformed Label column. 

MapKeyToValue would take columns with DataViewKeyType and convert them back to their original values

In [16]:
var newPipeline = mlContext.Transforms.Conversion.MapKeyToValue(new[]
{
    new InputOutputColumnPair("LabelOriginalValue","Label"),
    new InputOutputColumnPair("PredictedLabelOriginalValue", "PredictedLabel")

});

In [17]:
var transformedData = newPipeline.Fit(dataWithPredictions).Transform(dataWithPredictions);

In [19]:
// Let's iterate over first 5 items.
transformedData = mlContext.Data.TakeRows(transformedData, 5);
mlContext.Data.CreateEnumerable<TransformedData>(transformedData, reuseRowObject: false)

index,Label,PredictedLabel,LabelOriginalValue,PredictedLabelOriginalValue
0,1,1,AA,AA
1,2,2,BB,BB
2,3,3,CC,CC
3,4,4,DD,DD
4,1,1,AA,AA
